## Motion Heatmap

In [3]:
import numpy as np
import cv2
import copy
import argparse

def motion_heatmap():
    cap = cv2.VideoCapture("Vmart_front.mp4")
    fgbg = cv2.bgsegm.createBackgroundSubtractorMOG() #create a mask

    first_iteration_indicator = 1
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if (first_iteration_indicator == 1):
            first_frame = copy.deepcopy(frame)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            height, width = gray.shape[:2]
            accum_image = np.zeros((height, width), np.uint8)
            first_iteration_indicator = 0
        else:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            fgmask = fgbg.apply(gray)
            maxValue = 1
            thresh = 2
            maxValue = 2
            ret, th1 = cv2.threshold(fgmask, thresh, maxValue, cv2.THRESH_BINARY) # remove small movements
            accum_image = cv2.add(accum_image, th1) # makes an accumulation image
            
        color_image = im_color = cv2.applyColorMap(accum_image, cv2.COLORMAP_JET) # Color map applied to image (visualize motion)
        result_overlay = cv2.addWeighted(frame, 0.85, color_image, 0.25, 0)
        cv2.imshow('frame', result_overlay)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


def main():
    motion_heatmap()

if __name__=='__main__':
    main()

## In-Out Footfall Counter

In [5]:
from mylib.centroidtracker import CentroidTracker
from mylib.trackableobject import TrackableObject
from imutils.video import VideoStream
from imutils.video import FPS
from mylib import thread
import time
import numpy as np
import argparse, imutils
import time, dlib, cv2, datetime

# total_count=[]

t0 = time.time()
Thread = True
def chkSegment(slope,x,y,x1,y1):
    val = (y >= (slope*x) + (y1 - (slope*x1)))
    if val == True:
        return 'in'
    else:
        return 'out'
    
def run():
    CLASSES = ["person"]
    net = cv2.dnn.readNetFromCaffe('MobileNetSSD_deploy.prototxt', 'MobileNetSSD_deploy.caffemodel')
    vs = cv2.VideoCapture("demo.mp4")
    W = None
    H = None
    ct = CentroidTracker(maxDisappeared=40, maxDistance=50)
    trackers = []
    trackableObjects = {}

    totalFrames = 0
    totalDown = 0
    totalUp = 0
    x_new = []
    get_total = []
    get_time = []
    empty=[]
    empty1=[]

    fps = FPS().start()

    while True:
        ret, frame = vs.read()
        if frame is None:
            break

        frame = imutils.resize(frame, width = 500)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        if W is None or H is None:
            (H, W) = frame.shape[:2]

        status = "Waiting"
        rects = []

        if totalFrames % 2 == 0:
            status = "Detecting"
            trackers = []

            blob = cv2.dnn.blobFromImage(frame, 0.007843, (W, H), 127.5)
            net.setInput(blob)
            detections = net.forward()

            for i in np.arange(0, detections.shape[2]):
                confidence = detections[0, 0, i, 2]

                if confidence > 0.7:
                    idx = int(detections[0, 0, i, 1])

                    box = detections[0, 0, i, 3:7] * np.array([W, H, W, H])
                    (startX, startY, endX, endY) = box.astype("int")

                    tracker = dlib.correlation_tracker()
                    rect = dlib.rectangle(startX, startY, endX, endY)
                    tracker.start_track(rgb, rect)
                    trackers.append(tracker)

        else:  
            for tracker in trackers:
                status = "Tracking"

                tracker.update(rgb)
                pos = tracker.get_position()

                startX = int(pos.left())
                startY = int(pos.top())
                endX = int(pos.right())
                endY = int(pos.bottom())

                rects.append((startX, startY, endX, endY))

        x1 = (int)(0)
        y1 = (int)(H//1.8)
        x2 = (int)(W)
        y2 = (int)(H//1.8)
        cv2.line(frame, (x1, y1), (x2, y2), (0, 0, 0), 3)
        slope =   (y2 - y1) / (x2  - x1)
        
        objects = ct.update(rects)

        for (objectID, centroid) in objects.items():
            to = trackableObjects.get(objectID, None)

            if to is None:
                to = TrackableObject(objectID, centroid)

            else:
                y = [c[1] for c in to.centroids]
                x = [c[0] for c in to.centroids]
                init_direction  = chkSegment(slope, np.mean(x), np.mean(y) ,x1, y1)
                curr_direction  = chkSegment(slope, centroid[0], centroid[1], x1, y1)
                to.centroids.append(centroid)  
                
                if not to.counted:
                    if slope <= 0 and init_direction  == 'out' and curr_direction == 'in':
                        totalUp += 1
                        empty.append(totalUp)
                        to.counted = True

                    if slope <= 0 and init_direction  == 'in' and curr_direction == 'out':
                        totalDown += 1
                        empty1.append(totalDown)
                        if sum(x) >= 10:
                            cv2.putText(frame, "-ALERT: People limit exceeded-", (10, frame.shape[0] - 80),
                                cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255), 2)
                            to.counted = True
                    x_new = []
                    x_new.append(len(empty)-len(empty1))

            trackableObjects[objectID] = to
            text = "ID {}".format(objectID)
            cv2.rectangle(frame, (centroid[0] - 15, centroid[1] - 25), (centroid[0] + 30, centroid[1] - 5), (255,255,255), -1)
            cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            cv2.circle(frame, (centroid[0], centroid[1]), 4, (255, 0, 0), -1)
            cv2.circle(frame, (centroid[0], centroid[1]), 2, (255, 255, 255), -1)

        info = [
        ("Enter", totalUp),
        ("Exit", totalDown),
        ("Status", status),
        ]

        info2 = [
        ("Total people inside", x_new),
        ]

        for (i, (k, v)) in enumerate(info):
            text = "{}: {}".format(k, v)
            cv2.putText(frame, text, (10, H - ((i * 20) + 20)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        for (i, (k, v)) in enumerate(info2):
            text = "{}: {}".format(k, v)
            cv2.putText(frame, text, (265, H - ((i * 20) + 60)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        cv2.imshow("Real-Time Monitoring/Analysis Window", frame)
        key = cv2.waitKey(1) & 0xFF

        if key == ord("q"):
            break
            
        totalFrames += 1
        fps.update()
        if x_new:
            get_total.append(x_new[0])
    
    fps.stop()
    print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
    return get_total
    
    if Thread:
        vs.release()

    cv2.destroyAllWindows()


In [4]:
people_count = run()

In [6]:
cv2.destroyAllWindows()